In [34]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [39]:
mol = 'SO2'
mol_path = f'./{mol}'
tables = '../tables'

In [21]:
# Constants
pi = np.pi
h_k = 3.15775e5  # h_bar/K value (in Sec.K)

In [22]:
# Read input data
n_in = float(input("Give wavelength (in nm) of incident light:  "))
w_in = 45.613245464 / n_in  # in a.u
temp = float(input("Give the temp(in K): "))

Give wavelength (in nm) of incident light:   532
Give the temp(in K):  300


In [57]:
# Read autocorrelation data
# ac_data = np.loadtxt(f"{tables}/{mol}_acorr_sep.csv")
ac_data = np.loadtxt(f"./tables/{mol}_acorr_sep.csv")
# ac_data = pd.read_csv(f'./tables/{mol}_acorr.csv')
# ac_data
tot_struct = len(ac_data)
# dt = ac_data[1, 0] - ac_data[0, 0]
dt = 0.0014

In [51]:
dt

0.0014

In [6]:
# Calculate Raman spectrum
with open("Raman_11_model", "w") as outfile:
    for k in tqdm(range(-tot_struct // 2, tot_struct // 2 + 1)):
        sum_fourier_cos = 0.0
        sum_fourier_sin = 0.0

        if k != 0:
            f = k / (tot_struct * dt)
            wavenum = 33.333333 * f  # wavenumber in cm-1
            w = 1.5204417e-4 * f  # in a.u.

            for i in range(tot_struct):
                sum_fourier_cos += ac_data[i, 1] * np.cos(2 * pi * i * k / tot_struct)
                sum_fourier_sin += ac_data[i, 1] * np.sin(2 * pi * i * k / tot_struct)

            integration = np.sqrt(sum_fourier_cos**2 + sum_fourier_sin**2)
            exp_part = np.exp(-h_k * w / temp)
            r = ((w_in - w)**4 / (w * (1 - exp_part))) * integration

            outfile.write(f"{wavenum} {r}\n")

KeyboardInterrupt: 

In [58]:
# Vectorized functions for calculating Fourier sums
cos_func = np.vectorize(np.cos)
sin_func = np.vectorize(np.sin)

with open(f"./tables/{mol}_raman.csv", "w") as outfile:
    for k in tqdm(range(-tot_struct // 2, tot_struct // 2 + 1)):

        # Calculate wavenumber and frequency
        f = k / (tot_struct * dt)
        wavenum = 33.333333 * f  # wavenumber in cm-1
        w = 1.5204417e-4 * f  # in a.u.

        # Calculate Fourier sums using vectorized functions
        sum_fourier_cos = np.sum(cos_func(2 * np.pi * np.arange(tot_struct) * k / tot_struct) * ac_data[:, 1])
        sum_fourier_sin = np.sum(sin_func(2 * np.pi * np.arange(tot_struct) * k / tot_struct) * ac_data[:, 1])

        # Calculate integration, exponential part, and Raman intensity
        integration = np.sqrt(sum_fourier_cos**2 + sum_fourier_sin**2)
        exp_part = np.exp(-h_k * w / temp)
        r = ((w_in - w)**4 / (w * (1 - exp_part))) * integration

        # Write Raman intensity to file
        outfile.write(f"{wavenum},{r}\n")


 50%|███████▉        | 8271/16552 [03:06<02:33, 54.01it/s]/tmp/ipykernel_9387/1279666852.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  r = ((w_in - w)**4 / (w * (1 - exp_part))) * integration
100%|███████████████| 16552/16552 [06:16<00:00, 44.00it/s]
